In [1]:
#off-policy型モンテカルロ法

import numpy as np
import copy

#0～399のsanumberに対して、状態行動の割り当て
def getStateAction(sanumber):
    
    if sanumber % 2==1:
        hit=True
    else:
        hit=False
        
    if sanumber>=200:
        usableA=True
    else:
        usableA=False
        
    n=sanumber // 2
    n=n % 100
    
    sumCards=(n % 10)+12
    dealerCard=(n // 10)+2
    
    sa=[sumCards, dealerCard, usableA, hit]
    return sa

#状態ナンバー(0～199)
def getStateNumber(sumCards, dealerCard, usableA):
    n=(sumCards-12)+10*(dealerCard-2)
    if usableA:
        n+=100
    return n

def pickCard():
    #1～13の整数
    c=np.random.randint(1,14)
    if c>10:
        c=10
    elif c==1:
        c=11
    return c


def playerAct(sanumber):
    #経由した状態行動対の番号
    #---off-policyではパスが必要なのでリスト---
    sanumbers=[sanumber]
    #sanumbers=set([sanumber])
    
    sa=getStateAction(sanumber)
    
    sumCards=sa[0]
    dealerCard=sa[1]
    usableA=sa[2]
    hit=sa[3]

    while(hit):
        c=pickCard()
        sumCards+=c
        if c==11:
            usableA=True
        if sumCards>21 and usableA:
            sumCards-=10
            usableA=False
        if sumCards>21:
            break
            
        snumber=getStateNumber(sumCards, dealerCard, usableA)
        #乱数選択
        r=np.random.rand()
        if r<piH[snumber]:
            hit=True
            sanumber=snumber*2+1
        else:
            hit=False
            sanumber=snumber*2
        
        #---リスト---
        sanumbers.append(sanumber)
        #sanumbers.add(sanumber)
        
    result=[sumCards,dealerCard,sanumbers]
    return result
        
def dealerAct(dealerCard):
    usableA=False
    if dealerCard==11:
        usableA=True
    s=dealerCard

    while(s<17):
        c=pickCard()
        s+=c
        if c==11:
            usableA=True
        if s>21 and usableA:
            s-=10
            usableA=False
            
    return s

def getReturn(psum, dsum):
    if psum>21:
        r=-1
    elif dsum>21:
        r=1
    elif psum>dsum:
        r=1
    elif dsum>psum:
        r=-1
    else:
        r=0
        
    return r


#+++問題となる地点までゲームを進行し、状態番号を取得+++
def prepare():
    #ディーラー
    d=pickCard()
    #プレーヤー
    s=0
    usableA=False
    while s<12:
        c=pickCard()
        s+=c
        if s>21: #11からAを引いた時のみ22→12(この場合、Aの有効無効はそれまでの状態を引き継ぐ)
            s-=10
        elif c==11:
            usableA=True
    
    snumber=getStateNumber(s,d,usableA)
    return snumber

#+++各状態から方策に従い行動を決定+++
def decideAction(snumber, piH):
    sanumber=snumber*2
    r=np.random.rand()
    #乱数選択
    if r<piH[snumber]:
        sanumber+=1
    return sanumber

In [14]:
Q=np.random.rand(200,2)*2-1 #Qの初期化　[-1,1]の乱数
piH=np.ones(200)*0.5 #方策の初期化：piHの確率でHit ---挙動方策---
piT=np.ones(200)*0.5 #---ターゲット方策　初期値は等確率---
#---Cは確率比の和---
C=np.zeros(400).reshape(200,2)
n=1000 #試行セット数
e=0.1 #εの値。最適でない行動が選ばれる確率はe/2

'''
returns=[]
for i in range(400):
    ret=[] #appendはリストの方が速い？numPy配列は平均値計算が速いがAppendが遅い
    returns.append(ret)    
''' 

table=np.zeros((n,3))

#エピソード400回を１セットとして、nセットのシミュレーション
for i in range(n):
    
    piBfr=copy.deepcopy(piH)
    QBfr=copy.deepcopy(Q)
        
    for j in range(400):
        #+++初期状態を開始点探査でなくゲームの最初からランダム選択+++
        snum_init=prepare()        
        sanum_init=decideAction(snum_init, piH)
        result=playerAct(sanum_init)
        #result=playerAct(j)

        psum=result[0]
        sanumbers=result[2]
        dsum=0
        if psum<=21:
            dealerCard=result[1]
            dsum=dealerAct(dealerCard)
        ret=getReturn(psum,dsum)
        
        #---価値関数更新---
        W=1
        for sanumber in reversed(sanumbers):
            s=sanumber // 2
            a=sanumber % 2
            C[s,a]+=W
            Q[s,a]+=W/C[s,a]*(ret-Q[s,a])
            if Q[s,1]>Q[s,0]:                
                piT[s]=1
            elif Q[s,1]<Q[s,0]:
                piT[s]=0
            else:
                piT[s]=0.5
                
            if a==0:
                pt=1-piT[s] #ターゲット方策
                pb=1-piH[s] #挙動方策
            else:
                pt=piT[s]
                pb=piH[s]

            if (piT[s]==0 and a==1) or (piT[s]==1 and a==0):
                break
            
            W=W*pt/pb #algorithmでは分子１だけどタイ(0.5)があるので
            
        '''for sanumber in sanumbers:
            rlist=returns[sanumber]
            rlist.append(ret)
            ave=sum(rlist)/len(rlist)
            s=sanumber // 2
            a=sanumber % 2
            Q[s,a]=ave
            #+++方策更新+++            
            if Q[s,1]>Q[s,0]:#Hitの価値が高い時
                piH[s]=1-e/2
                #piH[s]=1
            else:
                piH[s]=e/2
                #piH[s]=0
        '''
    difPi=piH-piBfr
    difQ=Q-QBfr
    
    table[i,0]=i
    table[i,1]=np.count_nonzero(difPi)
    table[i,2]=int(np.linalg.norm(difQ)*10000)    

In [18]:
#結果の評価

print(Q)
print(piT) #---ターゲット方策---
#print(piH) 

matrixA=np.zeros((11,11))
matrixN=np.zeros((11,11))
valueA=np.zeros((11,11))
valueN=np.zeros((11,11))

for i in range(10):
    matrixA[i+1,0]=i+12 #プレーヤー
    matrixN[i+1,0]=i+12 #プレーヤー
    valueA[i+1,0]=i+12 #プレーヤー
    valueN[i+1,0]=i+12 #プレーヤー
    matrixA[0,i+1]=i+2 #ディーラー
    matrixN[0,i+1]=i+2 #ディーラー
    valueA[0,i+1]=i+2 #プレーヤー
    valueN[0,i+1]=i+2 #プレーヤー

for i in range(200):
    sa=getStateAction(i*2)
    h=piT[i]
    #h=piH[i]
    if h>0.5:
        optAct='H'
        v=Q[i,1]
    else:
        optAct='S'
        v=Q[i,0]
    sa[3]=optAct
    sa.insert(0,i)
    print(sa)
        
    player=i%10
    dealer=(i//10)%10
    usableA=i//100
    if usableA==1:
        matrixA[player+1,dealer+1]=h
        valueA[player+1,dealer+1]=int(v*1000)
    else:
        matrixN[player+1,dealer+1]=h
        valueN[player+1,dealer+1]=int(v*1000)
    
print('With A')
print(matrixA)
print('Without A')
print(matrixN)

print('Value with A')
print(valueA)
print('Value without A')
print(valueN)



[[  0.00000000e+00   0.00000000e+00   1.45718000e+05]
 [  1.00000000e+00   0.00000000e+00   1.05614000e+05]
 [  2.00000000e+00   0.00000000e+00   7.84760000e+04]
 ..., 
 [  9.97000000e+02   0.00000000e+00   4.31000000e+02]
 [  9.98000000e+02   0.00000000e+00   3.69000000e+02]
 [  9.99000000e+02   0.00000000e+00   2.57000000e+02]]
[[ -2.88329519e-01  -2.30948226e-01]
 [ -2.83965015e-01  -2.69298763e-01]
 [ -2.93788472e-01  -3.69603785e-01]
 [ -2.59216590e-01  -4.55079272e-01]
 [ -2.97583972e-01  -4.69340974e-01]
 [ -1.36555891e-01  -4.88123515e-01]
 [  1.33648729e-01  -6.27033792e-01]
 [  4.01323706e-01  -6.99692308e-01]
 [  6.36950904e-01  -8.27500000e-01]
 [  8.78531073e-01  -1.00000000e+00]
 [ -2.42265032e-01  -2.20436280e-01]
 [ -2.50148544e-01  -3.00606729e-01]
 [ -2.80641466e-01  -3.59544160e-01]
 [ -2.88582903e-01  -4.22028353e-01]
 [ -2.46506428e-01  -4.69005848e-01]
 [ -1.09806224e-01  -5.26506024e-01]
 [  1.71513353e-01  -6.37369392e-01]
 [  4.13298566e-01  -7.23922548e-01]
 [